# Aprendizaje basado en instancias

## Procesado de los datos

El conjunto de datos sobre cáncer de mama está incluido en *Scikit-learn*, se obtiene usando la función `load_breast_cancer` incluida en la librería `sklearn.datasets`. Este conjunto de datos contiene 569 ejemplos con 30 características sobre clasificaciones de cáncer de mama, con dos clasificaciones posibles.

In [80]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

Este conjunto de datos es un diccionario con varios campos:
* `data`: Es el conjunto de datos, se trata de un array en el que cada componente es un array con las características de cada instancia.
* `target`: Es el conjunto de valores de clasificación para cada instancia. Es un array del mismo tamaño que `data`, en el que se indica el valor de clasificación de cada instancia, en el mismo orden en que éstas se encuentran en el array `data`.
* `DESCR`: Es una descripción del conjunto de datos.
* `target_names`: Es un array con los nombres de cada valor de clasificación.
* `feature_names`: Es un array con los nombres de cada característica.

Almacenamos los datos en las variables `X_data`, `y_data`, `X_names` e `y_names`.

In [81]:
X_data, y_data, X_names, y_names = \
    cancer.data, cancer.target, cancer.feature_names, cancer.target_names

## Ejemplos aislados (*outliers*)

Un ejemplo aislado (*outlier* en inglés) es un ejemplo de entrenamiento que está rodeado por ejemplos que no pertenecen a su misma clase. La presencia de ejemplos aislados en un conjunto de entrenamiento puede deberse a:

* Un error en los datos
* Una cantidad insuficiente de ejemplos de la clase de los ejemplos aislados
* La existencia de características que no se están teniendo en cuenta
* Clases desbalanceadas 

En el contexto de la clasificación basada en instancias mediante el algoritmo **k**-*NN*, los ejemplos aislados generan ruido que disminuye el rendimiento del clasificador. Dados dos números naturales, $k$ y $r$, con $k>r$, decimos que un ejemplo es ($k$,$r$)-aislado si en sus $k$ vecinos más cercanos hay más de $r$ ejemplos que no pertenecen a su misma clase.

En este ejercicio vamos a identificar ejemplos ($k$,$r$)-aislados en el conjunto de datos y vamos a comparar el rendimiento del clasificador **K**-*NN* con y sin estos ejemplos. Para ello vamos a utilizar la clase `NearestNeighbors` de *Scikit-learn*, incluida en la librería `sklearn.neighbors`.

In [82]:
from sklearn.neighbors import NearestNeighbors

## Contenido del ejercicio

El ejercicio consiste en

* Investigar la clase `NearestNeighbors` de *Scikit-learn* y cómo se usa para obtener los vecinos más cercanos a un ejemplo dado dentro de un conjunto de datos, con respecto a una medida de distancia.
* Definir la función `buscaOutliers` que dados dos números naturales `k` y `r`, devuelve la lista de los índices de los ejemplos del conjunto de entrenamiento sobre el cáncer que son (`k`,`r`)-aislados. Es decir, la lista de los índices de los ejemplos del conjunto de entrenamiento `X_data` para los que en sus `k` vecinos más cercanos hay más de `r` ejemplos con una clasificación distinta a la del propio ejemplo.
* Construir un segundo conjunto de datos `Xo_data` obtenido a partir del inicial eliminando todos los ejemplos (5,3)-aislados.
* Construir dos modelos de decisión (para valores de `p` y `k` coherentes con la búsqueda de ejemplos aislados), uno para cada conjunto de datos considerado `X_data` y `Xo_data`, realizando en ambos casos una separación del $30$% de datos de prueba y $70$% de datos de entrenamiento.
* Comparar el rendimiento de ambos modelos sobre su correspondiente conjunto de prueba.

El **desarrollo tiene que estar razonado**, indicando en cada apartado qué se está haciendo, **demostrando así el conocimiento adquirido en este módulo**. ¿Qué conclusiones puedes sacar de lo aprendido sobre aprendizaje basado en instancias?

## Apartado 1

La clase **NearestNeighbors** posee los métodos necesarios para hallar la serie de vecinos de un miembro del conjunto. 
Básicamente crearemos un objeto de este tipo mediante su constructor, que no tiene parámetros obligatorios, pero con el cual podemos definir:

* **n_neighbors**: Por defecto = 5. Es el número de vecinos que serán determinantes en la búsqueda, el **k** de nuestro algoritmo **knn**.
* **radius**: Por defecto = 1.0. Lo usamos si queremos que el radio de distancia respecto al miembro del conjunto sea determinante y no lo sea únicamente el número de vecinos sin importar su distancia.
* **algorithm**: Para definir qué algoritmo queremos utilizar. Por defecto es 'auto', que determina el algoritmo automáticamente en base a los datos de aprendizaje.
* **otros**: El contructor de NearestNeighbors puede utilizar otros parámetros como **metric** o **p**, ya visto anteriormente en el módulo para seleccionar el método de medición de distancias. 

Para el presente ejercicio sólo usaremos **n_neighbors** para definir nuestra **k**.

In [83]:
from sklearn.neighbors import NearestNeighbors

Otro aspecto importante de la clase **NearestNeighbors** serán sus métodos **fit()** y **kneighbors()**.

* **fit()**: A este método le pasaremos nuestro conjunto de datos completo y nos servirá para entrenar el algortimo.
* **kneighbors()**: Con este método podemos encontrar los vecinos del miembro que pasamos como argumento. Pasamos el conjunto de datos, el k deseado y por último especificamos si queremos que se añadan las distancias al array resultante.

Veamos un ejemplo simple:

In [84]:
import numpy as np

#Nuestro conjunto de datos
samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]

#Definimos el objeto con k=2 y radio=0.4
neigh = NearestNeighbors(n_neighbors=2, radius=0.4)
#Entrenamos
neigh.fit(samples)

#¿Cuáles son los 2 vecinos más cercanos de [0, 0, 1.3] en samples = [[0, 0, 2], [1, 0, 0], [0, 0, 1]]?
neigh.kneighbors([[0, 0, 1.3]], 2, return_distance=False)

array([[2, 0]], dtype=int64)

Como podemos observar, el método devuelve un conjunto de índices. Los índices 2 y 0 nos muestran que [0, 0, 1] y [0, 0, 2] son los 2 vecinos más cercanos a [0, 0, 1.3].

## Apartado 2

Vamos a crear el método **buscaOutliers**, que encuentra los elementos **(𝑘,𝑟)-aislado** del conjunto de datos.

En este método vamos a recorrer **X_data** de principio a fin. Para cada uno de sus elemento comprobaremos de entre sus **k-vecinos** cuáles son de un tipo diferente al del elemento de la instancia actual. En el ejercicio se ha tenido en cuenta que **kneighbors** devuelve también el propio elemento de la instancia puesto que su distancia es 0. Por ello, de manera interna, usaremos k+1 al buscar los vecinos y posteriormente borraremos el primer elemento de la lista, siendo éste el propio elemento de la instancia, que es considerado trivial para determinar la información que deseamos encontrar.

In [85]:
import numpy as np

def buscaOutliers(k,r):
    #Defino NearestNeighbors para k+1 vecinos y lo entreno con todo mi conjunto de datos
    neigh = NearestNeighbors(n_neighbors=k+1)
    neigh.fit(X_data, y_data)
    outliers_array = []
    for objetivo in range(len(X_data)):
        #Encuentro los k+1 vecinos del actual objetivo
        vecinos_posible_outlier = neigh.kneighbors([X_data[objetivo]],return_distance=False)
        #Elimino el primer elemento del array, es decir, el objetivo en sí mismo
        vecinos_posible_outlier2 = np.delete(vecinos_posible_outlier,0)
        #Recojo en un array el conjunto de datos de los vecinos
        vecinos_data = y_names[y_data[vecinos_posible_outlier2]]
        #Compruebo de entre los vecinos, cuáles son del mismo tipo que el objetivo. 
        #Si el total es menor que k-r, objetivo es outlier
        if(np.count_nonzero(vecinos_data == y_names[y_data[objetivo]]) < (k-r)):
            outliers_array.append(objetivo)        
    return outliers_array

## Apartado 3

Busquemos los **outliers** de nuestro conjunto de datos:

In [86]:
outliers = buscaOutliers(5,3)
print(outliers)

[3, 14, 26, 38, 39, 41, 86, 91, 92, 99, 135, 146, 157, 190, 194, 209, 215, 297, 363, 379, 385, 430, 465, 491, 536]


Éste sería el conjunto de índices indicando todos los elementos pertenecientes a nuestro datos que son sonsiderados **outliers** para **k=5** y **r=3**.

Para contruir **Xo_data** basta con eliminar los elementos **outliers** del conjunto **X_data**.

In [87]:
Xo_data = np.delete(X_data,outliers, 0)

También debemos eliminar los elementos de **y_data**.

In [88]:
yo_data = np.delete(y_data,outliers, 0)

Con esto queda listo nuestro conjunto de datos **Xo_data**.

## Apartado 4

Vamos a crear los modelos de decisión de **X_data** y **Xo_data**. Para ello usaremos la librería **model_selection**.

In [89]:
from sklearn.model_selection import train_test_split

Empezamos con el conjunto **x_data**, haremos una división 70/30 en los datos de entrenamiento/prueba. Bastará con que especifiquemos el valor de **0.30** para test_size.

In [90]:
X_train, X_test, y_train, y_test = \
  train_test_split(X_data,y_data,test_size = 0.30,
                   random_state=4861,stratify=y_data)

A continuación usaremos la clase **KNeighborsClassifier** para la construcción del modelo. Ésto nos permitirá utilizar los métodos necesarios.

In [91]:
from sklearn.neighbors import KNeighborsClassifier

Usamos **k=5** tal y como hicimos al buscar los outliers. También respetaremos el método de medición de distancias, distancias euclideas, usado anteriormente.

In [92]:
knn1 = KNeighborsClassifier(n_neighbors=5,p=2)

Entrenamos el modelo con los datos de training.

In [93]:
knn1.fit(X_train,y_train)

KNeighborsClassifier()

Para comprobar que el modelo funciona de forma correcta podemos definir el siguiente código:

In [94]:
knn1.predict(X_test)

array([1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1])

Siendo **knn1.predict(X_test)** la predicción de resultados que genera nuestro modelo en base a los datos de training. 
Podemos comparar este resultado con los datos de prueba **y_test**, caso ideal:

In [95]:
y_clas = knn1.predict(X_test)
y_clas[y_clas != y_test]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0])

Como podemos observar, la predicción ha dado un resultado distinto al preteneciente a **y_test** en 10 elementos.
Sabiendo la cantidad de elementos de **y_test** con la función **shape()** y sabiendo que la predicción coincide en esta cantidad restando 10 elementos podemos concluir lo siguiente:

In [96]:
rendimiento = (y_test.shape[0]-10)/y_test.shape[0]
rendimiento

0.9415204678362573

Es decir, modelo para **X_data** tiene un rendimiento del **0.94%** aproximadamente.
Estas últimas operaciones pueden resumirse en la función **score()** como vemos a continuación:

In [97]:
knn1.score(X_test,y_test)

0.9415204678362573

Actuaremos de forma análoga para construir el modelo de **Xo_data**.

In [98]:
Xo_train, Xo_test, yo_train, yo_test = \
  train_test_split(Xo_data,yo_data,test_size = 0.30,
                   random_state=4861,stratify=yo_data)

In [99]:
knn2 = KNeighborsClassifier(n_neighbors=5,p=2)
knn2.fit(Xo_train,yo_train)

KNeighborsClassifier()

In [100]:
knn2.score(Xo_test,yo_test)

1.0

Un rendimiento de **1.0** indica que no se han cometido errores en la predicción, es decir, tenemos un rendimiento del **100%**.

## Apartado 5

Por último vamos a comparar el rendimiento de los modelos:

In [101]:
rendimiento_X_data = knn1.score(X_test,y_test)
rendimiento_Xo_data = knn2.score(Xo_test,yo_test)

print('Rendimiento de X_data:',rendimiento_X_data)
print('Rendimiento de Xo_data:',rendimiento_Xo_data)
print('Diferencia de porcentaje', rendimiento_Xo_data-rendimiento_X_data)

Rendimiento de X_data: 0.9415204678362573
Rendimiento de Xo_data: 1.0
Diferencia de porcentaje 0.05847953216374269


El modelo de **Xo_data** es casi 6 puntos mayor en rendimiento al modelo de **X_data**, podemos concluir que el procesamiento de **outliers** ha sido muy fructífero en esta ocasión puesto que hemos obtenido un segundo modelo, no sólo más eficiente, sino del **100%** de rendimiento posible.

## Conclusiones

El aprendizaje basado en instancias es muy útil cuando tenemos datos estáticos, tipos diferenciados y poco ruido. El algoritmo **Knn** no es un concepto nuevo al menos respecto a ingeniería de computadores pero sí lo es el uso de éste con tuplas complejas de datos y el tener en cuenta la influencia de **outliers**. Es por ello que las principales conclusiónes de lo aprendido son ganar la capacidad de sacar provecho al algoritmo en casos reales y poder conocer de forma desgranada el porqué del rendimiento de un modelo concreto.